In [8]:
#2nd Approach with Tensorflow libary
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
data=pd.read_csv("Sentiment.csv")
#keeping only text and sentiment columns
data = data[['text','sentiment','candidate']]
data.shape

(13871, 3)

In [24]:
texts = data.text.values
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(texts)
encoded_docs = tokenizer.texts_to_sequences(texts)
padded_sequence = pad_sequences(encoded_docs, maxlen = 200)
sentiment_lable = data.sentiment.factorize()

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SpatialDropout1D, Embedding
embedding_vector_length = 64
vocab_size = 10000
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length = 200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(20, dropout = 0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 200, 64)           640000    
                                                                 
 spatial_dropout1d_9 (Spatia  (None, 200, 64)          0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 20)                6800      
                                                                 
 dropout_8 (Dropout)         (None, 20)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 21        
                                                                 
Total params: 646,821
Trainable params: 646,821
Non-trainable params: 0
_______________________________________________

In [50]:
history = model.fit(padded_sequence, sentiment_lable[0], validation_split=0.2, epochs=2, batch_size=64)

Epoch 1/2
174/174 [==============================] - 66s 344ms/step - loss: -1.7499 - accuracy: 0.1593 - val_loss: -4.5168 - val_accuracy: 0.1686
Epoch 2/2
174/174 [==============================] - 59s 341ms/step - loss: -4.0979 - accuracy: 0.1593 - val_loss: -6.4892 - val_accuracy: 0.1686


In [51]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted Label : ",sentiment_lable[1][prediction])

In [52]:
example1 = ["RT @DanScavino: #GOPDebate @realDonaldTrump delivered the highest ratings in the history of presidential debates"]
example2 = ["RT @NancyOsborne180: Last night's debate proved it! #GOPDebate #BATsAsk @BadassTeachersA #TBATs"]
example3 = ["Going on #MSNBC Live with @ThomasARoberts around 2 PM ET.  #GOPDebate"]
predict_sentiment(example1)
predict_sentiment(example2)
predict_sentiment(example3)

1/1 [==============================] - 1s 545ms/step
Predicted Label :  Positive
1/1 [==============================] - 0s 46ms/step
Predicted Label :  Positive
1/1 [==============================] - 0s 54ms/step
Predicted Label :  Positive
